# Topic Modeling

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

## Topic Modeling - Attempt #1 (All Text)

In [1]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

,aarp,abandon,abc,abdomen,abdominal,ability,able,abort,abortion,abraham,...,zipline,ziplining,zipper,zoloft,zombie,zone,zoo,zoom,zorb,zuko
ali,0,0,1,0,0,0,2,0,0,0,...,0,0,0,0,1,0,0,0,0,0
amy,0,0,0,0,0,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
andrew,0,0,0,0,0,0,2,0,0,0,...,1,1,0,0,0,2,0,0,1,0
anthony,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
beth,0,0,0,0,1,0,3,0,2,0,...,0,0,1,0,0,0,0,0,0,0
bill,1,0,0,0,0,1,1,0,2,0,...,0,0,0,0,0,1,0,2,0,0
chelsea,0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,1,0,0,0,0
joe,0,0,0,1,0,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
mae,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,0
matt,0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [2]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,ali,amy,andrew,anthony,beth,bill,chelsea,joe,mae,matt,michelle,mike,neal,roseanne,sammy,sarah,shane,stavros,taylor,whitney
aarp,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
abandon,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
abc,1,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0
abdomen,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
abdominal,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [5]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [6]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.014*"right" + 0.012*"guy" + 0.009*"think" + 0.009*"people" + 0.009*"got" + 0.008*"yeah" + 0.008*"want" + 0.007*"thing" + 0.007*"time" + 0.006*"gonna"'),
 (1,
  '0.007*"think" + 0.007*"want" + 0.007*"right" + 0.007*"said" + 0.006*"say" + 0.006*"got" + 0.006*"yeah" + 0.006*"time" + 0.006*"guy" + 0.006*"theyre"')]

In [7]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.010*"right" + 0.008*"people" + 0.008*"guy" + 0.007*"got" + 0.007*"joke" + 0.007*"yeah" + 0.007*"say" + 0.006*"thing" + 0.005*"think" + 0.005*"going"'),
 (1,
  '0.014*"right" + 0.011*"guy" + 0.009*"yeah" + 0.009*"think" + 0.009*"want" + 0.008*"got" + 0.008*"gonna" + 0.007*"people" + 0.007*"time" + 0.007*"theyre"'),
 (2,
  '0.009*"guy" + 0.009*"think" + 0.009*"want" + 0.008*"right" + 0.008*"people" + 0.008*"fucking" + 0.007*"got" + 0.007*"time" + 0.007*"going" + 0.006*"thing"')]

In [8]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.012*"guy" + 0.011*"right" + 0.009*"got" + 0.009*"people" + 0.008*"want" + 0.008*"say" + 0.007*"think" + 0.007*"thing" + 0.007*"yeah" + 0.007*"theyre"'),
 (1,
  '0.008*"want" + 0.008*"think" + 0.008*"gonna" + 0.007*"right" + 0.007*"time" + 0.007*"shit" + 0.007*"people" + 0.006*"god" + 0.006*"okay" + 0.005*"fucking"'),
 (2,
  '0.019*"right" + 0.013*"think" + 0.012*"guy" + 0.012*"yeah" + 0.009*"fuckin" + 0.008*"got" + 0.008*"gonna" + 0.008*"women" + 0.008*"want" + 0.007*"people"'),
 (3,
  '0.009*"yeah" + 0.008*"feel" + 0.007*"want" + 0.007*"time" + 0.007*"okay" + 0.007*"got" + 0.007*"think" + 0.007*"theyre" + 0.006*"right" + 0.006*"guy"')]

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [9]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [10]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,transcript
ali,lady and gentlemen please welcome the stage a...
amy,yes thank you much chicago thank you use s...
andrew,hey andrew santino and here hometown chic...
anthony,thank you thank you thank you san francisco th...
beth,beth stellings standup comedy special you did...
bill,lady and gentlemen bill burr all right than...
chelsea,air date november many trial and tribul...
joe,what the fuck boston goddamn thank you go...
mae,jesus mamma mia mae jeez hey sorry marshm...
matt,his second hourlong comedy special matthew st...


In [11]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
ali,lady gentlemen stage ali thank hello na shit c...
amy,thank something lot city people chicago night ...
andrew,hey santino chicago illinois comedy take city ...
anthony,thank thank people gon tape francisco city wor...
beth,stellings comedy victoria dayton ohio show rel...
bill,lady gentlemen bill thank right year half nice...
chelsea,air date trial tribulation point year life sta...
joe,boston goddamn love goddamn see people time wo...
mae,jesus mamma mia mae jeez hey rubber band thank...
matt,comedy matthew stephen rife rife present candi...


In [12]:
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = list(text.ENGLISH_STOP_WORDS.union(add_stop_words))  # Convert set to list

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn



,abc,abdomen,ability,abort,abortion,abraham,abscess,absence,absolute,absurdity,...,zealand,zelda,zeppelin,zero,zion,zip,zipper,zone,zoo,zoom
ali,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
amy,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
andrew,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,2,0,0
anthony,0,0,0,0,2,0,0,0,0,1,...,7,0,0,0,0,0,0,0,0,0
beth,0,0,0,0,2,0,0,0,0,2,...,0,0,0,0,1,0,1,0,0,0
bill,0,0,1,0,2,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,2
chelsea,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
joe,0,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
mae,0,0,0,0,1,0,0,2,0,1,...,0,0,0,0,0,0,0,1,0,1
matt,0,0,0,0,0,0,0,0,2,2,...,0,0,0,0,0,0,0,1,0,0


In [13]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [14]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.014*"guy" + 0.013*"thing" + 0.012*"man" + 0.010*"women" + 0.009*"men" + 0.008*"day" + 0.007*"life" + 0.007*"lot" + 0.006*"way" + 0.006*"shit"'),
 (1,
  '0.013*"thing" + 0.011*"guy" + 0.008*"life" + 0.008*"shes" + 0.008*"way" + 0.008*"cause" + 0.008*"theyre" + 0.007*"year" + 0.007*"kind" + 0.007*"gon"')]

In [15]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.015*"thing" + 0.013*"women" + 0.013*"guy" + 0.012*"men" + 0.009*"kind" + 0.008*"man" + 0.008*"cause" + 0.008*"fuck" + 0.008*"life" + 0.007*"woman"'),
 (1,
  '0.014*"guy" + 0.013*"thing" + 0.010*"man" + 0.009*"life" + 0.008*"year" + 0.008*"day" + 0.008*"theyre" + 0.007*"way" + 0.007*"joke" + 0.007*"friend"'),
 (2,
  '0.010*"thing" + 0.009*"cause" + 0.008*"way" + 0.007*"life" + 0.007*"women" + 0.007*"shes" + 0.007*"guy" + 0.007*"day" + 0.007*"year" + 0.006*"friend"')]

In [16]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.013*"thing" + 0.012*"women" + 0.012*"men" + 0.011*"guy" + 0.010*"cause" + 0.009*"theyre" + 0.009*"kind" + 0.009*"life" + 0.009*"way" + 0.008*"shes"'),
 (1,
  '0.013*"thing" + 0.011*"guy" + 0.010*"day" + 0.010*"joke" + 0.008*"year" + 0.008*"life" + 0.008*"friend" + 0.007*"way" + 0.007*"cause" + 0.006*"man"'),
 (2,
  '0.015*"thing" + 0.013*"guy" + 0.009*"shes" + 0.009*"women" + 0.008*"life" + 0.008*"woman" + 0.008*"way" + 0.007*"man" + 0.007*"kind" + 0.006*"sex"'),
 (3,
  '0.013*"guy" + 0.013*"man" + 0.010*"thing" + 0.009*"shit" + 0.008*"cause" + 0.006*"theyre" + 0.006*"day" + 0.006*"gon" + 0.006*"year" + 0.006*"lot"')]

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [17]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [18]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
ali,lady gentlemen stage ali wong hello welcome th...
amy,much chicago thank something lot comic city pe...
andrew,hey santino hometown chicago illinois standup ...
anthony,thank san francisco thank good people gon tape...
beth,beth stellings comedy special want victoria da...
bill,lady gentlemen bill right thank right standin ...
chelsea,air date many trial tribulation point crazy ye...
joe,fuck boston goddamn good fuck love goddamn goo...
mae,jesus mamma mia mae jeez hey good rubber band ...
matt,second hourlong comedy special matthew stephen...


In [19]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,abc,abdomen,abdominal,ability,able,abort,abortion,abraham,abscess,absence,...,zero,zion,zip,zipper,zoloft,zombie,zone,zoo,zoom,zuko
ali,1,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
amy,0,0,0,0,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
andrew,0,0,0,0,2,0,0,0,0,0,...,1,0,0,0,0,0,2,0,0,0
anthony,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
beth,0,0,1,0,3,0,2,0,0,0,...,0,1,0,1,0,0,0,0,0,0
bill,0,0,0,1,1,0,2,0,0,0,...,0,0,0,0,0,0,1,0,2,0
chelsea,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
joe,0,1,0,0,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mae,0,0,0,0,0,0,1,0,0,2,...,0,0,0,0,0,0,1,0,1,0
matt,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [20]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [21]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.009*"fuck" + 0.007*"dude" + 0.007*"fuckin" + 0.006*"dad" + 0.004*"white" + 0.004*"dog" + 0.003*"ta" + 0.003*"room" + 0.003*"black" + 0.003*"dick"'),
 (1,
  '0.005*"fuck" + 0.005*"dick" + 0.003*"white" + 0.003*"relationship" + 0.003*"sex" + 0.003*"dog" + 0.003*"gay" + 0.003*"marry" + 0.003*"phone" + 0.003*"period"')]

In [22]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.010*"fuck" + 0.008*"dude" + 0.006*"dog" + 0.005*"dick" + 0.005*"gay" + 0.005*"black" + 0.005*"white" + 0.004*"dad" + 0.003*"cat" + 0.003*"let"'),
 (1,
  '0.014*"fuckin" + 0.006*"fuck" + 0.005*"dad" + 0.005*"dick" + 0.004*"white" + 0.004*"lesbian" + 0.004*"jenny" + 0.004*"sex" + 0.004*"end" + 0.004*"ta"'),
 (2,
  '0.004*"fuck" + 0.004*"dad" + 0.004*"pregnant" + 0.004*"room" + 0.003*"ta" + 0.003*"sex" + 0.003*"story" + 0.003*"parent" + 0.003*"dead" + 0.003*"husband"')]

In [23]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.004*"fuck" + 0.004*"pregnant" + 0.004*"marry" + 0.004*"white" + 0.004*"dog" + 0.003*"husband" + 0.003*"jenny" + 0.003*"relationship" + 0.003*"liberal" + 0.003*"party"'),
 (1,
  '0.010*"fuckin" + 0.008*"fuck" + 0.007*"dad" + 0.006*"dick" + 0.005*"dude" + 0.005*"sex" + 0.004*"relationship" + 0.004*"ta" + 0.003*"story" + 0.003*"room"'),
 (2,
  '0.013*"fuck" + 0.006*"cat" + 0.006*"dog" + 0.006*"gay" + 0.006*"dude" + 0.005*"black" + 0.005*"bar" + 0.004*"room" + 0.004*"bathroom" + 0.004*"story"'),
 (3,
  '0.008*"white" + 0.008*"dude" + 0.007*"dad" + 0.007*"period" + 0.006*"penis" + 0.005*"news" + 0.005*"dick" + 0.004*"fck" + 0.004*"abortion" + 0.004*"gross"')]

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [24]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.010*"fuckin" + 0.006*"dad" + 0.005*"fuck" + 0.004*"dude" + 0.004*"room" + 0.003*"end" + 0.003*"white" + 0.003*"sex" + 0.003*"story" + 0.003*"true"'),
 (1,
  '0.010*"fuck" + 0.007*"dick" + 0.006*"cat" + 0.006*"sex" + 0.006*"dog" + 0.006*"dude" + 0.005*"relationship" + 0.004*"gay" + 0.004*"liberal" + 0.003*"porn"'),
 (2,
  '0.006*"period" + 0.006*"penis" + 0.005*"dick" + 0.005*"abortion" + 0.005*"birth" + 0.004*"dad" + 0.004*"palestinian" + 0.004*"white" + 0.003*"otter" + 0.003*"gross"'),
 (3,
  '0.009*"fuck" + 0.006*"black" + 0.006*"dude" + 0.005*"gay" + 0.005*"white" + 0.004*"party" + 0.004*"dog" + 0.004*"dick" + 0.004*"red" + 0.004*"ta"')]

These four topics look pretty decent. Let's settle on these for now.
* Topic 0: mom, parents
* Topic 1: relationship/liberal
* Topic 2: guns/war/labour
* Topic 3: profanity

In [25]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

[(3, 'ali'),
 (1, 'amy'),
 (3, 'andrew'),
 (3, 'anthony'),
 (2, 'beth'),
 (0, 'bill'),
 (3, 'chelsea'),
 (1, 'joe'),
 (0, 'mae'),
 (3, 'matt'),
 (2, 'michelle'),
 (0, 'mike'),
 (1, 'neal'),
 (0, 'roseanne'),
 (2, 'sammy'),
 (0, 'sarah'),
 (0, 'shane'),
 (0, 'stavros'),
 (1, 'taylor'),
 (1, 'whitney')]

##### For a first pass of LDA, these kind of make sense to me, so we'll call it a day for now.
##### Topic 0: mom, parents [Bill,Mae,Mike,Roseanne,Sarah,Shane,Stavros]
##### Topic 1: husband, wife [Taylor,Whitney,Amy,Joe,Neal]
##### Topic 2: guns [Beth,Michelle,Sammy]
##### Topic 3: profanity [Ali,Andrew,Anthony,Chelsea,Matt]

### Assignment:
1. Try further modifying the parameters of the topic models above and see if you can get better topics.
2. Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if you can get better topics.

In [29]:
ldana = models.LdaModel(corpus=corpusna, num_topics=5, id2word=id2wordna, passes=100)
ldana.print_topics()

[(0,
  '0.007*"dad" + 0.005*"parent" + 0.004*"mail" + 0.004*"feeling" + 0.004*"room" + 0.004*"sex" + 0.003*"dead" + 0.003*"hot" + 0.003*"relationship" + 0.003*"story"'),
 (1,
  '0.009*"fuck" + 0.007*"dick" + 0.006*"sex" + 0.006*"dude" + 0.005*"dog" + 0.005*"cat" + 0.004*"relationship" + 0.004*"youve" + 0.004*"gay" + 0.004*"white"'),
 (2,
  '0.019*"fuckin" + 0.012*"fuck" + 0.008*"white" + 0.008*"black" + 0.006*"dick" + 0.006*"gay" + 0.006*"penis" + 0.005*"dad" + 0.005*"lesbian" + 0.004*"ta"'),
 (3,
  '0.006*"jenny" + 0.006*"fuck" + 0.005*"flag" + 0.005*"red" + 0.005*"dog" + 0.004*"phone" + 0.004*"relationship" + 0.004*"dude" + 0.003*"hate" + 0.003*"gay"'),
 (4,
  '0.006*"dad" + 0.006*"dude" + 0.005*"pregnant" + 0.005*"fuck" + 0.004*"ta" + 0.004*"white" + 0.004*"husband" + 0.004*"room" + 0.003*"news" + 0.003*"anthony"')]

#### Topic 0 -parents-mon/dad
#### Topic 1 -relationship
#### Topic 2 -race/orientation
#### Topic 3 -profanity
#### Topic 4 -husband/wife

In [30]:
# Let's create a function to pull out proper nouns and adjectives from a string of text
def propernouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NNP' or pos[:2] == 'JJ'or pos[:2]=='NNPS'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [31]:
# Apply the nouns function to the transcripts to filter only on nouns
data_pronouns_adj = pd.DataFrame(data_clean.transcript.apply(propernouns_adj))
data_pronouns_adj

,transcript
ali,wong hello welcome hello ten much exciting exc...
amy,much chicago comic last last crazy pregnant am...
andrew,hometown standup special want okay fair fine o...
anthony,san francisco good special san correct favorit...
beth,beth special want hilarious beth personal rela...
bill,right standin funny ive right denver legal fed...
chelsea,many crazy top special own dont last seriousne...
joe,fuck good fuck good happy good tense fight shi...
mae,good sure ill right nice okay mine hey nice be...
matt,second hourlong special provocative rifes inti...


In [32]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_pronouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_pronouns_adj.index
data_dtmna

,abc,abdominal,able,abraham,absolute,abstract,abyss,accent,acceptable,accessible,...,younger,youngest,yous,youth,youthful,youve,yucky,zoloft,zombie,zuko
ali,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
amy,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
andrew,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
beth,0,1,3,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
bill,0,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chelsea,0,0,3,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
joe,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mae,0,0,0,0,1,1,0,1,1,0,...,0,0,0,1,0,0,0,0,0,0
matt,0,0,3,0,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [34]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.017*"fuckin" + 0.009*"okay" + 0.007*"fuck" + 0.007*"hot" + 0.007*"dead" + 0.007*"ta" + 0.007*"weird" + 0.006*"dick" + 0.006*"white" + 0.005*"shit"'),
 (1,
  '0.016*"white" + 0.013*"black" + 0.009*"okay" + 0.008*"red" + 0.007*"true" + 0.006*"ta" + 0.005*"liberal" + 0.005*"high" + 0.005*"gross" + 0.005*"important"')]

In [36]:
# Let's start with 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=100)
ldana.print_topics()

[(0,
  '0.010*"okay" + 0.008*"pregnant" + 0.007*"hot" + 0.006*"ta" + 0.006*"true" + 0.005*"huge" + 0.005*"guy" + 0.005*"dead" + 0.005*"dick" + 0.005*"fuck"'),
 (1,
  '0.023*"fuckin" + 0.021*"white" + 0.015*"black" + 0.008*"liberal" + 0.008*"ta" + 0.008*"okay" + 0.006*"lesbian" + 0.006*"shit" + 0.006*"hot" + 0.006*"fuck"'),
 (2,
  '0.011*"red" + 0.009*"true" + 0.009*"white" + 0.008*"important" + 0.008*"okay" + 0.008*"god" + 0.007*"high" + 0.007*"fuck" + 0.006*"gay" + 0.006*"gross"')]

#### Topic 0 - Raunchy/profanity/Tale
#### Topic 1 - Political/Liberal/Woke
#### Topic 2 - faith